In [ ]:
%matplotlib qt
%gui qt5
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op
import matplotlib.pyplot as plt
import mne
import csv
import batch_average_erp as ba
from sklearn.utils import shuffle
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

### Importing preprocessed data - epochs

In [ ]:
overwrite = True
eeg_path = r'D:\Master_thesis\Pre-processed_data'
data_path_out = r'D:\Master_thesis\Scores\Binary classification\ERP_windows_minibatch'

### Windows ERP

In [ ]:
scores_all_best  = []

emotion_combinations = ["happy", "neutral"], ["happy", "angry"], ["happy", "surprised"], ["neutral", "angry"], ["neutral", "surprised"], ["angry", "surprised"]  
emo_comb_num = [1, 2, 3, 4, 5, 6]                        
# emo_comb_num = [1]


l1_ratio = [0.05, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
C = [0.01, 0.1, 0.2, 0.3, 0.5, 0.6, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 3, 4]
scoring = 'accuracy' 

n_split = np.arange(1, 6 , dtype = int)
# n_split = [1]

scores_all  = []
scores_CV = []

windows_time = ["P1", 0.08, 0.12], ["N170", 0.14, 0.20] ,["EPN", 0.15, 0.35], ["LPC", 0.40, 0.60]
win_comb_num = [1, 2, 3, 4]
# win_comb_num = [1]


# subject = 'VR2FEM_S01'

for subject in os.listdir(eeg_path):
    data_path = os.path.join(eeg_path, subject)
    #reading preprocessed data -epochs
    epochs_train_list = []
    epochs_test_list = []

    for items in n_split:
        epochs_train_name = os.path.join(f"{data_path}\{subject}-preprocessed_train_{items}-epo.fif")
        epochs_test_name = os.path.join(f"{data_path}\{subject}-preprocessed_test_{items}-epo.fif")
        epochs_train = mne.read_epochs(epochs_train_name, preload=True)
        epochs_test = mne.read_epochs(epochs_test_name, preload=True)
        epochs_train_list.append(epochs_train)
        epochs_test_list.append(epochs_test)

    for com_num in emo_comb_num:
        print(f"Number of emo combination: {com_num}")
        for windows in win_comb_num:
            wins = windows-1
            print(f"Window split: {windows_time[wins][0]}")
            scores_CV = []

            for l1 in l1_ratio:
                for cs in C:
                    print(f"Train/test split: {items}")
                    print(f"Emotion combination: {emotion_combinations[com_num-1]}")
                    print("Running reg. ratio for: ",l1)
                    print("Running reg. strength for: ",cs)   
                    clf = make_pipeline(
                    StandardScaler(),
                    LogisticRegression(max_iter = 4000, solver='saga',penalty='elasticnet', l1_ratio=l1, C=cs))

                    scores_all  = []

                    for split in n_split:
                        n = split-1 
                        print(f"Subject: {subject} Number of test/train split: {split} for L1 ratio : {l1} and strength: {cs}")

                        n_batch = 3
                        train_emotion_1 = epochs_train_list[n][emotion_combinations[com_num-1][0]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                        train_av_emotion_1 = np.mean(train_emotion_1, axis = 2)
                        train_minibatch_av_emotion_1 = ba.batch_average_erp(train_av_emotion_1, n_batch)


                        test_emotion_1 = epochs_test_list[n][emotion_combinations[com_num-1][0]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                        test_av_emotion_1 = np.mean(test_emotion_1, axis = 2)
                        test_minibatch_av_emotion_1 = ba.batch_average_erp(test_av_emotion_1, n_batch)

                        train_emotion_2 = epochs_train_list[n][emotion_combinations[com_num-1][1]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                        train_av_emotion_2 = np.mean(train_emotion_2, axis = 2)
                        train_minibatch_av_emotion_2 = ba.batch_average_erp(train_av_emotion_2, n_batch)


                        test_emotion_2 = epochs_test_list[n][emotion_combinations[com_num-1][1]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                        test_av_emotion_2 = np.mean(test_emotion_2, axis = 2)
                        test_minibatch_av_emotion_2 = ba.batch_average_erp(test_av_emotion_2, n_batch)



                        y_train_emotion_1 = np.zeros(len(train_minibatch_av_emotion_1))
                        y_test_emotion_1 = np.zeros(len(test_minibatch_av_emotion_1))

                        y_train_emotion_2 = np.ones(len(train_minibatch_av_emotion_2))
                        y_test_emotion_2 = np.ones(len(test_minibatch_av_emotion_2))

                        x_train_mini = np.append(train_minibatch_av_emotion_1, train_minibatch_av_emotion_2, axis = 0)
                        x_test_mini = np.append(test_minibatch_av_emotion_1, test_minibatch_av_emotion_2, axis = 0)

                        y_train_mini = np.append(y_train_emotion_1, y_train_emotion_2, axis = 0)
                        y_test_mini = np.append(y_test_emotion_1, y_test_emotion_2, axis = 0)



                        clf.fit(x_train_mini, y_train_mini)
                        clf.predict(x_test_mini)
                        score = clf.score(x_test_mini, y_test_mini)
                        scores_all.append(score)

                    print("----------------Ending Split ----------------")

                    scores_CV_mini = np.vstack(scores_all)
                    scores_avg_mini = np.mean(scores_CV_mini, axis=0)   


                scores_CV.append(scores_avg_mini)


            scores_avg_best = max(scores_CV)
            print(f"Maximum score: {scores_avg_best}")
            dir_save= os.path.join(data_path_out, subject)
            if not op.exists(dir_save):
                        os.makedirs(dir_save)
            score_file_name = f"{dir_save}\{subject}_{emotion_combinations[com_num-1][0]} vs {emotion_combinations[com_num-1][1]}_{windows_time[wins][0]}.csv"
            print(score_file_name)
            with open(score_file_name, "w") as file:
                writer = csv.writer(file)
                writer.writerow([scores_avg_best.item()])

